# Install

In [1]:
!pip install fredapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import

In [2]:
import requests
import sys
import json
import pandas as pd
import numpy as np
import fredapi as fa

from datetime import datetime
from datetime import timedelta

# Constants
Tweakable constants

In [3]:
OUTPUT_FILE = 'DATA.csv'

DATE_FROM = '2022-06-30T00:00:00'
DATE_TO = '2022-09-25T00:00:00'

# percentage of valid values per row
EMPTY_VALUES_THRESHOLD = 0.7

In [4]:
#EPIC_SNP500 = 'SPX'
#EPIC_SPDR_SNP500 = 'SPY'

EPIC_APPLE = 'AAPL'
EPIC_TESLA = 'TSLA'
EPIC_META = 'META'
EPIC_GOOGLE = 'GOOG'
EPIC_NETFLIX = 'NFLX'
EPIC_AMAZON = 'AMZN'
EPIC_NVIDIA = 'NVDA'
EPIC_INTEL = 'INTC'
EPIC_PAYPAL = 'PYPL'
EPIC_AMD = 'AMD'
EPIC_MICROSOFT = 'MSFT'
EPIC_IBM = 'IBM'

EPIC_USD_INDEX = 'DXY'
EPIC_USD_CAD = 'USDCAD'
EPIC_USD_JPY = 'USDJPY'
EPIC_USD_CHN = 'USDCNH'
EPIC_USD_EUR = 'EURUSD'
EPIC_USD_GBP = 'GBPUSD'

EPIC_VANGUARD_CONSUMER_DISCRETIONARY = 'VCR'
EPIC_CONSUMER_DISCRETIONARY_SELECT_SECTOR_SPDR_FUND = 'XLY'
EPIC_ISHARES_20_PLUS_YR_TREASURY_BOND = 'TLT'
EPIC_VOLATILITY_INDEX = 'VIX'
EPIC_SNP500 = 'US500'
EPIC_NASDAQ100 = 'US100'
EPIC_DJIA = 'US30'
EPIC_NASDAQ = 'NDAQ'
EPIC_NIKKEI225 = 'J225'
EPIC_RUSSEL2000 = 'RTY'
EPIC_DAX = 'DE40'
EPIC_HSI = 'HK50'
EPIC_AUS200 = 'AU200'
EPIC_SSE = 'SSE'
EPIC_FTSE100 = 'UK100'
EPIC_EU50 = 'EU50'
EPIC_CN50 = 'CN50'
EPIC_VANGUARD_TOTAL_STOCK_MARKET = 'VTI'
EPIC_VGT = 'VGT'
EPIC_ISHARES_MSCI_EMERGING_MARKETS = 'EEM'

EPIC_GOLD = 'GOLD'
EPIC_SILVER = 'SILVER'
EPIC_COPPER = 'COPPER'
EPIC_PLATINUM = 'PLATINUM'
EPIC_PALLADIUM = 'PALLADIUM'
EPIC_LEAD = 'LEAD'
EPIC_CRUDE_OIL = 'OIL_CRUDE'
EPIC_BRENT_OIL = 'OIL_BRENT'
EPIC_NATURAL_GAS = 'NATURALGAS'
EPIC_COFFEE_ARABICA = 'COFFEEARABICA'

FRED_10_YR_TREASURY_CONST_MATURITY_MINUS_2_YR = 'T10Y2Y'
FRED_10_YR_BREAKEVEN_INFLATION_RATE = 'T10YIE'
FRED_5_YR_BREAKEVEN_INFLATION_RATE = 'T5YIE'
FRED_5_YR_FORWARD_EXPECTED_INFLATION_RATE = 'T5YIFR'

In [5]:
STOCK_INTEREST = EPIC_APPLE

STOCK_LIST = [
   EPIC_APPLE,
   EPIC_TESLA,
   EPIC_META,
   EPIC_GOOGLE,
   EPIC_NETFLIX,
   EPIC_AMAZON,
   EPIC_NVIDIA,
   EPIC_INTEL,
   EPIC_AMD,
   EPIC_MICROSOFT,
   EPIC_IBM,

   #EPIC_VANGUARD_CONSUMER_DISCRETIONARY,
   EPIC_CONSUMER_DISCRETIONARY_SELECT_SECTOR_SPDR_FUND,
   EPIC_SNP500,
   EPIC_NASDAQ100,
   EPIC_DJIA,
   EPIC_NIKKEI225,
   EPIC_RUSSEL2000,
   EPIC_DAX,
   EPIC_HSI,
   EPIC_AUS200,
   EPIC_SSE,
   EPIC_FTSE100,
   EPIC_EU50,
   EPIC_CN50,
   EPIC_VANGUARD_TOTAL_STOCK_MARKET,
   EPIC_VGT,
   EPIC_ISHARES_MSCI_EMERGING_MARKETS,
   EPIC_ISHARES_20_PLUS_YR_TREASURY_BOND,
   EPIC_USD_INDEX,
   EPIC_USD_CAD,
   EPIC_USD_JPY,
   EPIC_USD_CHN,
   EPIC_USD_EUR,
   EPIC_USD_GBP,
   EPIC_GOLD,
   EPIC_SILVER,
   EPIC_COPPER,
   EPIC_PLATINUM,
   EPIC_PALLADIUM,
   EPIC_CRUDE_OIL,
   EPIC_BRENT_OIL,
   EPIC_NATURAL_GAS,
   EPIC_VOLATILITY_INDEX,
]

FRED_LIST = [
    FRED_10_YR_TREASURY_CONST_MATURITY_MINUS_2_YR,
    FRED_10_YR_BREAKEVEN_INFLATION_RATE,
    FRED_5_YR_BREAKEVEN_INFLATION_RATE,
    FRED_5_YR_FORWARD_EXPECTED_INFLATION_RATE,
]

In [6]:
DATE_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATE_DF_FORMAT = '%Y-%m-%d'

QUERY_NUM = 1000
QUERY_INTERVAL = 'DAY'

Capital.com API Constants

In [7]:
API_KEY = '<ENTER_API_KEY>'
API_LOGIN = '<ENTER_LOGIN>'
API_PASS = '<ENTER_PASSWORD>'

API_ENDPOINT = 'https://demo-api-capital.backend-capital.com'
API_PREFIX = '/api/v1'
API_BASE_URL = API_ENDPOINT + API_PREFIX

In [8]:
API_CONTENT_TYPE_KEY = 'Content-Type'
API_CONTENT_TYPE_VALUE = 'application/json'

FRED API Constants

In [9]:
fred = fa.Fred(api_key='6ffa4ce000638cd6532d02bfb42fedc7')

# API Path

In [10]:
PATH_API_SESSION = '/session'
PATH_API_MARKET = '/market'
PATH_API_PRICES = '/prices'
PATH_API_WATCHLISTS = '/watchlists'

# Helper Functions

In [11]:
def GetDateList(date_time_from, date_time_to):
  date_list = []
  while date_time_from <= date_time_to:
    no = date_time_from.weekday()

    # Only add weekday
    if no < 5:
      date_list.append(date_time_from.strftime(DATE_DF_FORMAT))

    date_time_from = date_time_from + timedelta(days=1)

  return date_list

def ConvertDictOfObjectToDictOfValue(map, field):
  ret = {}
  for key in map:
    ret[key] = map[key][field]
  
  return ret

# Create Session
Prepare header and body

In [12]:
create_session_payload = {
    'identifier': API_LOGIN,
    'password': API_PASS,
}

create_session_headers = {
    'X-CAP-API-KEY': API_KEY,
    API_CONTENT_TYPE_KEY: API_CONTENT_TYPE_VALUE,
}

Create Session

In [13]:
create_session_url = API_BASE_URL + PATH_API_SESSION
result = requests.post(create_session_url,
                       data=json.dumps(create_session_payload),
                       headers=create_session_headers)

Parse create session response

Extract required token

In [14]:
cst = result.headers['CST']
security_token = result.headers['X-SECURITY-TOKEN']

In [15]:
api_headers = {
    'CST': cst,
    'X-SECURITY-TOKEN': security_token,
}

# Get Market Prices
Get market prices from `STOCK_LIST`



In [16]:
def GetMarketPrices(api_headers, stock_interest, stock_list, query_interval, query_num, date_from, date_to):
  date_time_from = datetime.strptime(date_from, DATE_FORMAT)
  date_time_to = datetime.strptime(date_to, DATE_FORMAT)

  date_list = GetDateList(date_time_from, date_time_to)
  df = pd.DataFrame({
      'Date': date_list,
  })

  interest_price_map = DoGetStockPrice(stock_interest, query_interval, query_num, date_time_from, date_time_to, ProcessInterestStockPrices)
  AddInterestStockInfoToDatadrame(df, interest_price_map, date_list, 'Open')
  AddInterestStockInfoToDatadrame(df, interest_price_map, date_list, 'High')
  AddInterestStockInfoToDatadrame(df, interest_price_map, date_list, 'Low')
  AddInterestStockInfoToDatadrame(df, interest_price_map, date_list, 'Close')
  AddInterestStockInfoToDatadrame(df, interest_price_map, date_list, 'Volume')

  for epic in stock_list:
    # skip
    if epic == stock_interest:
      continue

    price_map = DoGetStockPrice(epic, query_interval, query_num, date_time_from, date_time_to, ProcessStockPrices)

    series = pd.Series(price_map, index=date_list)
    print(series.tolist())
    df[epic] = series.tolist()

  return df

def DoGetStockPrice(epic, query_interval, query_num, date_from, date_to, processor):
  date_time_current_from = date_from
  price_map = {}

  while date_time_current_from < date_to:
    date_time_current = date_time_current_from + timedelta(days=query_num - 1)
    if date_time_current > date_to:
      date_time_current = date_to
      
    api_params = {
      'resolution': query_interval,
      'max': query_num,
      'from': date_time_current_from.strftime(DATE_FORMAT),
      'to': date_time_current.strftime(DATE_FORMAT),
    }

    print("[DoGetStockPrice] " + epic + " | date from: " + api_params['from'] + " - date to: " + api_params['to'])
      
    get_prices_url = API_BASE_URL + PATH_API_PRICES + '/' + epic
    result = requests.get(get_prices_url, api_params, headers=api_headers)
    response = result.json()

    #print("[GetMarketPrices] response: ")
    #print(json.dumps(response))
  
    processor(response, price_map)

    date_time_current_from = date_time_current
    print("-----------------------------------------------------------------------------------------------------")

  return price_map

def AddInterestStockInfoToDatadrame(df, price_map, date_list, field):
  map = ConvertDictOfObjectToDictOfValue(price_map, field)
  series = pd.Series(map, index=date_list)
  df[field] = series.tolist()

def ProcessStockPrices(response, price_map):
  for price in response['prices']:
    cd = datetime.strptime(price['snapshotTimeUTC'], DATE_FORMAT)

    if 'ask' in price['closePrice']:
      price_map[cd.strftime(DATE_DF_FORMAT)] = price['closePrice']['ask']
    else:
      price_map[cd.strftime(DATE_DF_FORMAT)] = price['closePrice']['bid']

def ProcessInterestStockPrices(response, price_map):
  for price in response['prices']:
    cd = datetime.strptime(price['snapshotTimeUTC'], DATE_FORMAT)
    key = cd.strftime(DATE_DF_FORMAT)
    price_map[key] = {}
    if 'ask' in price['openPrice']:
      price_map[key]['Open'] = price['openPrice']['ask']
      price_map[key]['High'] = price['highPrice']['ask']
      price_map[key]['Low'] = price['lowPrice']['ask']
      price_map[key]['Close'] = price['closePrice']['ask']
    else:
      price_map[key]['Open'] = price['openPrice']['bid']
      price_map[key]['High'] = price['highPrice']['bid']
      price_map[key]['Low'] = price['lowPrice']['bid']
      price_map[key]['Close'] = price['closePrice']['bid']
      
    price_map[key]['Volume'] = price['lastTradedVolume']
      

In [17]:
df = GetMarketPrices(api_headers, STOCK_INTEREST, STOCK_LIST, QUERY_INTERVAL, QUERY_NUM, DATE_FROM, DATE_TO)

[DoGetStockPrice] AAPL | date from: 2022-06-30T00:00:00 - date to: 2022-09-25T00:00:00
-----------------------------------------------------------------------------------------------------
[DoGetStockPrice] TSLA | date from: 2022-06-30T00:00:00 - date to: 2022-09-25T00:00:00
-----------------------------------------------------------------------------------------------------
[224.34, 226.76, nan, 233.08, 231.58, 243.72, 250.77, 234.03, 232.17, 234.85, 239.49, 240.0, 241.4, 247.18, 251.32, 269.75, 272.0, 266.79, 262.34, 273.71, 284.39, 296.73, 298.0, 301.76, 307.86, 309.34, 288.34, 293.07, 284.0, 293.32, 288.24, 300.02, 309.68, 306.64, 303.17, 303.34, 296.69, 290.54, 296.67, 298.1, 296.01, 287.01, 286.21, 278.23, 272.26, 280.01, 270.11, nan, 274.07, 283.54, 290.07, 299.97, 304.74, 291.83, 304.41, 300.32, 303.4, 309.53, 307.51, 299.33, 304.56, 275.15]
[DoGetStockPrice] META | date from: 2022-06-30T00:00:00 - date to: 2022-09-25T00:00:00
---------------------------------------------------

In [18]:
df.head()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,GBPUSD,GOLD,SILVER,COPPER,PLATINUM,PALLADIUM,OIL_CRUDE,OIL_BRENT,NATURALGAS,VIX
0,2022-06-30,137.47,138.40,133.81,137.08,119113.0,224.34,161.44,109.62,177.02,...,1.21612,1806.65,20.277,3.6915,902.14,1939.94,104.77,108.66,5.657,28.74
1,2022-07-01,135.51,139.15,135.46,139.05,122379.0,226.76,160.09,109.17,180.18,...,1.21054,1811.12,19.889,3.6155,897.82,1972.83,106.69,110.74,5.645,28.01
2,2022-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.21143,1809.58,20.001,3.5875,891.26,1939.93,108.54,112.72,5.849,27.81
3,2022-07-05,139.21,141.79,136.96,141.46,142460.0,233.08,168.30,113.85,186.04,...,1.19521,1769.81,19.254,3.4435,872.90,1946.99,99.28,101.83,5.624,28.25
4,2022-07-06,141.90,144.15,140.25,143.00,144194.0,231.58,169.92,115.52,184.27,...,1.19230,1740.21,19.193,3.4375,862.58,1922.80,95.76,98.89,5.545,27.66


In [19]:
orig_df = df

In [20]:
df = df.dropna(thresh=int(len(STOCK_LIST) * EMPTY_VALUES_THRESHOLD + 1))
df = df.replace(np.nan, '#NA')

In [21]:
df.head()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,GBPUSD,GOLD,SILVER,COPPER,PLATINUM,PALLADIUM,OIL_CRUDE,OIL_BRENT,NATURALGAS,VIX
0,2022-06-30,137.47,138.40,133.81,137.08,119113.0,224.34,161.44,109.62,177.02,...,1.21612,1806.65,20.277,3.6915,902.14,1939.94,104.77,108.66,5.657,28.74
1,2022-07-01,135.51,139.15,135.46,139.05,122379.0,226.76,160.09,109.17,180.18,...,1.21054,1811.12,19.889,3.6155,897.82,1972.83,106.69,110.74,5.645,28.01
3,2022-07-05,139.21,141.79,136.96,141.46,142460.0,233.08,168.30,113.85,186.04,...,1.19521,1769.81,19.254,3.4435,872.90,1946.99,99.28,101.83,5.624,28.25
4,2022-07-06,141.90,144.15,140.25,143.00,144194.0,231.58,169.92,115.52,184.27,...,1.19230,1740.21,19.193,3.4375,862.58,1922.80,95.76,98.89,5.545,27.66
5,2022-07-07,143.61,146.59,142.88,145.99,67581.0,243.72,171.75,119.30,188.73,...,1.20297,1742.08,19.259,3.5450,880.88,2002.01,100.33,102.65,6.159,27.47


# Get FRED Data

In [22]:
def AppendFREDData(fred_list, df):
  df_date_list = df['Date'].tolist()

  for symbol in fred_list:
    item = fred.get_series(symbol)

    item.index = item.index.strftime(DATE_DF_FORMAT)
    add = item[[i in df_date_list for i in item.index]]
    add = add.reindex(df_date_list, fill_value='#NA')
    df[symbol] = add.tolist()

  return df

In [23]:
df = AppendFREDData(FRED_LIST, df)

In [24]:
df.tail()

,Date,Open,High,Low,Close,Volume,TSLA,META,GOOG,NFLX,...,PLATINUM,PALLADIUM,OIL_CRUDE,OIL_BRENT,NATURALGAS,VIX,T10Y2Y,T10YIE,T5YIE,T5YIFR
57,2022-09-19,149.60,155.02,148.73,155.02,174953.0,309.53,148.52,104.82,244.50,...,925.52,2226.86,85.39,90.80,7.950,25.58,-0.46,2.34,2.44,2.24
58,2022-09-20,153.88,158.11,153.13,156.51,193568.0,307.51,146.07,102.38,243.30,...,927.90,2171.09,83.96,90.02,7.815,27.42,-0.39,2.40,2.48,2.32
59,2022-09-21,156.48,158.73,152.25,152.49,296918.0,299.33,140.86,99.71,234.94,...,911.00,2157.13,82.77,89.00,7.799,28.64,-0.51,2.38,2.47,2.29
60,2022-09-22,153.89,154.97,150.94,154.93,267191.0,304.56,143.87,101.27,238.85,...,916.36,2162.99,84.30,90.38,7.791,27.54,-0.41,2.41,2.45,2.37
61,2022-09-23,152.19,152.39,148.59,150.67,258020.0,275.15,140.21,99.53,226.89,...,872.61,2077.40,79.16,85.68,7.045,30.04,-0.51,2.37,2.38,2.36


# Save to CSV

In [25]:
df.to_csv(OUTPUT_FILE, index=False)